In [1]:
import torch


In [2]:
words = open('Data\\names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y



In [5]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [26]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn(27,2, generator=g)
W1 = torch.randn(6,100, generator=g)
b1=torch.randn(100, generator=g)
W2 = torch.randn(100, 27, generator=g)
b2 = torch.randn(27, generator=g)
parameters = [W1, b1, W2, b2, C]

for p in parameters:
    p.requires_grad= True


In [31]:
sum =0
for p in parameters:
    sum+=p.nelement()
sum

3481

In [28]:
def training_loop(parameters, X,Y):
   learning_rate = 1e-2
   for i in range(1,1000):
    W1, b1, W2, b2, C = parameters
    for p in parameters:
       p.grad= None
    emb = C[X]
    h = emb.view(-1, 6) @ W1 + b1
    logits = h@W2 +b2
    loss = torch.nn.functional.cross_entropy(logits, Y)
    loss.backward()
    for p in parameters:
        p.data -= learning_rate*p.grad
    
    print (loss)

training_loop(parameters, Xtr, Ytr)




tensor(37.9292, grad_fn=<NllLossBackward0>)
tensor(36.6747, grad_fn=<NllLossBackward0>)
tensor(35.4981, grad_fn=<NllLossBackward0>)
tensor(34.3866, grad_fn=<NllLossBackward0>)
tensor(33.3336, grad_fn=<NllLossBackward0>)
tensor(32.3347, grad_fn=<NllLossBackward0>)
tensor(31.3878, grad_fn=<NllLossBackward0>)
tensor(30.4920, grad_fn=<NllLossBackward0>)
tensor(29.6473, grad_fn=<NllLossBackward0>)
tensor(28.8551, grad_fn=<NllLossBackward0>)
tensor(28.1162, grad_fn=<NllLossBackward0>)
tensor(27.4279, grad_fn=<NllLossBackward0>)
tensor(26.7836, grad_fn=<NllLossBackward0>)
tensor(26.1749, grad_fn=<NllLossBackward0>)
tensor(25.5953, grad_fn=<NllLossBackward0>)
tensor(25.0409, grad_fn=<NllLossBackward0>)
tensor(24.5092, grad_fn=<NllLossBackward0>)
tensor(23.9984, grad_fn=<NllLossBackward0>)
tensor(23.5073, grad_fn=<NllLossBackward0>)
tensor(23.0350, grad_fn=<NllLossBackward0>)
tensor(22.5812, grad_fn=<NllLossBackward0>)
tensor(22.1460, grad_fn=<NllLossBackward0>)
tensor(21.7296, grad_fn=<NllLoss

KeyboardInterrupt: 

tensor(55.4190)

2